In [ ]:
# sacado de la documentación, yo no lo he ejecutado.
# %pip install llama-index-llms-openai llama-index-embeddings-openai

# estos son los módulos que tengo instalados
# pip install llama-index
# pip install llama-index-embeddings-huggingface
# pip install llama-index-llms-ollama
# pip install matplotlib

In [3]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "TU KEY"
# openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI

In [6]:
from llama_index.core.llama_dataset import download_llama_dataset, LabelledRagDataset

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/paul_graham/rag_dataset.json") en principio no lo necesitamos.
documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/paul_graham/source_files").load_data()

In [7]:
node_parser = SentenceSplitter(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)

# si se quiere ahorrar dinero y algo de tiempo, quizá aquí coger los n primeros nodos.

In [8]:
# by default, the node ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

In [9]:
vector_index = VectorStoreIndex(nodes)
retriever = vector_index.as_retriever(similarity_top_k=2)

In [10]:
# pequeña prueba si queremos ver los resultados del retriever con una query.
# no influye en la evaluación y se puede saltar.

from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = retriever.retrieve("What did the author do growing up?")

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** node_0<br>**Similarity:** 0.8071635617672756<br>**Text:** What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines â CPU, disk drives, printer, card reader â sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them...<br>

**Node ID:** node_57<br>**Similarity:** 0.8042717199680722<br>**Text:** I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with some problem involving continuations while I watched them play in the tide pools. It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.

In the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.

In the fall of 2019, Bel was finally finished. Like McCarthy's original Lisp, it's a spec rather than an implementation, although like McCarthy's Lisp it's a spec expressed as code.

Now that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writing essays through 2020, but I also start...<br>

In [11]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)

In [12]:
llm = OpenAI(model="gpt-4")

# si se quiere ahorrar dinero, quizá usar aquí un modelo más barato.

In [2]:
# NO EJECUTAR SI vas a cargar el dataset de un archivo guardado (más abajo), ya que esta celda gasta dinero y en ese caso no es necesaria.

qa_dataset = generate_question_context_pairs(
    nodes, llm=llm, num_questions_per_chunk=2
)

In [17]:
queries = qa_dataset.queries.values()
print(list(queries)[2])

"Describe the limitations of the IBM 1401 computer as mentioned in the text and explain how the advent of microcomputers changed the computing landscape."


In [15]:
# [optional] save
# es conveniente guardarlo si para generarlo se ha usado un modelo que cuesta dinero.
# el mismo sirve para probar muchos retrievers sin necesidad de gastar más.
qa_dataset.save_json("eval_data/generated/pg_eval_retriever_dataset.json")

In [16]:
# [optional] load
qa_dataset = EmbeddingQAFinetuneDataset.from_json("eval_data/generated/pg_eval_retriever_dataset.json")

In [18]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [19]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

Query: In the context provided, the author mentions his early experiences with programming. Describe the computing environment he worked in, including the type of computer, the programming language used, and the method of input and output.
Metrics: {'hit_rate': 1.0, 'mrr': 1.0, 'precision': 0.5, 'recall': 1.0, 'ap': 1.0, 'ndcg': 0.6131471927654584}



In [20]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [23]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [24]:
display_results("top-2 eval", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-2 eval,0.428571,0.345238,0.214286,0.428571,0.345238,0.225062
